<a href="https://colab.research.google.com/github/fhswf/ki-wir/blob/main/VAEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Variational Autoencoder


In [11]:
# Setup on Colab
!pip install gradio &> /dev/null
!pip install pytorch_lightning &> /dev/null
!if [ ! -e ki_wir ]; then git clone https://github.com/fhswf/ki-wir.git ki_wir; else cd ki_wir; git pull; fi

Already up to date.


In [12]:
import gradio as gr
import torch
import numpy as np
import ki_wir.models.vanilla_vae as vae
import ki_wir.models.experiment as experiment

from PIL import Image
from torchvision import transforms
import torchvision.utils as vutils

In [38]:
device = torch.device("cuda:0")
params={"in_channels": 3, "latent_dim": 128, "img_size": 64}
model = vae.VanillaVAE(**params)
exp = experiment.VAEXperiment(model, params)
exp.load_from_checkpoint("ki_wir/pretrained/vanilla.ckpt", vae_model=model, params=params).to(device)



def reconstruct(original):
  SetRange = transforms.Lambda(lambda X: 2 * X - 1.)
  img = Image.fromarray(original)
  img = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
        ])(img)

  orig = transforms.ToPILImage(mode='RGB')(img)
  img = SetRange(img)

  #img = torch.moveaxis(img, 0, -1)
  img = torch.unsqueeze(img.cuda(), 0)
  dec = exp.model.generate(img, latent_dim=128)
 
  dec = torch.squeeze(dec[0], 0)
  dec = transforms.Lambda(lambda X: 0.5 * (X + 1.))(dec)

  return [orig, transforms.ToPILImage(mode='RGB')(dec)]

In [14]:
test_label = ""
exp.curr_device = device
samples = exp.model.sample(144, device)
vutils.save_image(samples.cpu().data,
                  f"sample.png",
                  normalize=True,
                  nrow=12)

In [40]:
out1 = gr.outputs.Image(type="auto", label="original")
out2 = gr.outputs.Image(type="auto", label="reconstructed")
iface = gr.Interface(fn=reconstruct, inputs="image", outputs=[out1, out2]).launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://56166.gradio.app

This share link will expire in 72 hours. To get longer links, send an email to: support@gradio.app


KeyboardInterrupt: ignored